# How to Run DISC on the Iris Dataset 

First, we load iris dataset and convert it into a sparse binary matrix format.

In [1]:
import numpy as np
from sklearn.datasets import load_iris

iris = load_iris()
data = iris.data.round(0)
labels = list(iris.target.astype(int))

data[:,0] -= min(data[:,0])
for c in [1, 2, 3]:
    data[:,c] -= min(data[:,c]) - max(data[:,c - 1]) - 1
    
data = list(data.astype(int))

ModuleNotFoundError: No module named 'sklearn'

1. Discover a patternset for the complete data
2. Discover patterns and characterize classes in terms of common or characteristic patterns
3. Discover the pattern-composition, i.e. the partitioning of iris and its characteristic patternsets

In [2]:
import disc

r1 = disc.discover_patternset(data)
r2 = disc.characterize_partitions(data, labels)
r3 = disc.discover_composition(data)

NameError: name 'dat' is not defined

In [3]:
%matplotlib notebook

import matplotlib
import matplotlib.pyplot as plt

matplotlib.style.use('ggplot')

l1 = r1['objective'] / r1['initial_objective']
l2 = r2['objective'] / r2['initial_objective']
l3 = r3['objective'] / r3['initial_objective']
plt.bar([0, 1, 2], [l1, l2, l3])

NameError: name 'r1' is not defined